## Tool calling and Multi-Modal

In [13]:
from dataclasses import dataclass, field
from typing import List, Optional
import base64
from io import BytesIO
from autogen_core.base import MessageContext, AgentId, AgentInstantiationContext
from autogen_core.components import DefaultTopicId, RoutedAgent, default_subscription, message_handler
from autogen_core.components.code_executor import CodeExecutor, extract_markdown_code_blocks
from autogen_core.components.models import (
    AssistantMessage,
    ChatCompletionClient,
    LLMMessage,
    SystemMessage,
    UserMessage,
)
import tempfile
from autogen_core.components.tool_agent import ToolAgent, tool_agent_caller_loop
from autogen_core.application import SingleThreadedAgentRuntime
from autogen_core.components.models import OpenAIChatCompletionClient
import random

from autogen_core.base import CancellationToken
from autogen_core.components.tools import FunctionTool, ToolSchema
from autogen_core.components._image import Image
from typing_extensions import Annotated
from pydantic import BaseModel
import json


# Load .env

In [ ]:
%load_ext dotenv
%dotenv .env

## What is tool calling?

Tool call is essentially a function that can be executed by LLM or agent. A tool is consisted by two parts
- schema: the function signature and the description of the function
- function: the actual implementation of the function

Where the schema is used by LLM to generate the function arguments. When the function arguments are received, the corresponding function will be invoked with the given arguments.

## Define a tool

In AutoGen, to define a tool, simply create a python function and use `FunctionTool` class to wrap it. The `FunctionTool` class will automatically generate the schema for the function.

In [ ]:
async def get_weather(city: str, state: str) -> str: # is the return type matter here?
# get the weather for a city and state
    print(f"Getting the weather for {city}, {state}")
    return f"The weather in {city}, {state} is sunny."

# Create a get_weather tool
get_weather_tool = FunctionTool(get_weather, description="Get the weather for a city and state.")
get_weather_tool.schema

## Create AutoGen Assistant Agent

The assistant will be invoked when it receives a `Message` from agent runtime. Then it will use LLM to generate a response message and publish it back to the agent runtime.

The assistant has `default_subscription`, which means it subscribles to messages with all topics from the agent runtime.

The assistant agent uses `tool_agent_caller_loop` utility function to send a message to the tool agent and get the response.

In [15]:
class Message(BaseModel):
    text: str
    source: Optional[str] = None

@default_subscription
class WeatherAssistant(RoutedAgent):
    def __init__(self, model_client: ChatCompletionClient, get_weather_tool: FunctionTool) -> None:
        super().__init__("An agent with tools")
        self._system_messages: List[LLMMessage] = [SystemMessage("You are a helpful AI assistant.")]
        self._model_client = model_client
        self._get_weather_tool = get_weather_tool
        self._tools = [get_weather_tool]

    @message_handler
    async def handle_text_message(self, message: Message, ctx: MessageContext) -> None:
        # Create a session of messages.
        session: List[LLMMessage] = [UserMessage(content=message.text, source="user")]
        # Run the caller loop to handle tool calls.
        # the tool_agent_caller_loop function will call the tool agent to handle the tool calls if exists.
        completion = await self._model_client.create(
            session,
            tools=self._tools,
            cancellation_token=ctx.cancellation_token,
        )
        print(f"### {self.type}: \n{completion.content}")
        if not isinstance(completion.content, list):
            await self.publish_message(
                Message(text=completion.content),
                DefaultTopicId(),
            )
        # run tool call
        for tool_call in completion.content:
            arguments = tool_call.arguments
            arguments = json.loads(tool_call.arguments)
            result = await self._get_weather_tool.run_json(arguments, ctx.cancellation_token)
            reply = Message(text=self._get_weather_tool.return_value_as_string(result))
            print(f"### {self.type}: \n{reply.text}")
            await self.publish_message(
                reply,
                DefaultTopicId(),
            )

## Add assistant agent to the runtime

In [ ]:
# Create an local embedded runtime.
import os;
runtime = SingleThreadedAgentRuntime()
await WeatherAssistant.register(
        runtime,
        "assistant",
        lambda: WeatherAssistant(
            OpenAIChatCompletionClient(
                model="gpt-4o-mini",
                # get api key from env:OPENAI_API_KEY
                api_key=os.environ.get("OPENAI_API_KEY"),
            ),
            get_weather_tool,
        ),
    )

## Send a message to agent

In [ ]:
# Start the runtime and publish a message to the assistant.
runtime.start()
await runtime.publish_message(
    Message(text = "Hello, I am Geeno, what's the weather in New York"), DefaultTopicId()
)

await runtime.stop_when_idle()

## Conclusion

Congratulations! You have successfully created a simple tool agent and an assistant agent. And how to enable multi-modal chat in AutoGen.

## What we have learned
- How to create a tool agent
- How to equip the agent with an assistant
- How to enable multi-modal chat in assistant agent